In [1]:
import pennylane as qml
from pennylane import numpy as pnp
import numpy as np
from qiskit.quantum_info import SparsePauliOp
from scipy.optimize import minimize, differential_evolution
from scipy.stats.qmc import Halton
import os
import json
from susy_qm import calculate_Hamiltonian

import itertools

In [104]:
potential = 'DW'
cutoff = 16
shots = 1024

In [105]:
#calculate Hamiltonian and expected eigenvalues
H = calculate_Hamiltonian(cutoff, potential)

eigenvalues, eigenvectors = np.linalg.eig(H)
min_index = np.argmin(eigenvalues)
min_eigenvalue = eigenvalues[min_index]
min_eigenvector = np.asarray(eigenvectors[:, min_index])

#create qiskit Hamiltonian Pauli string
hamiltonian = SparsePauliOp.from_operator(H)
num_qubits = hamiltonian.num_qubits

In [106]:
min_3_ev = eigenvalues.argsort()[:3]
min_eigenvector = np.asarray(eigenvectors[:, min_3_ev[1]])

In [107]:
min_eigenvalue

np.complex128(0.8915993623272622+0j)

In [108]:
dev = qml.device("default.qubit", wires=num_qubits, shots=shots)

@qml.qnode(dev)
def basis_circuit(basis_state):

    qml.BasisState(basis_state, wires=range(num_qubits))

    return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))

In [109]:
basis_list = list(itertools.product([0,1], repeat=num_qubits))
basis = []
for _ in range(20):
    energies = []
    for bs in basis_list:
        energies.append(basis_circuit(bs))

    print(basis_list[np.argmin(energies)], energies[np.argmin(energies)])
    basis.append([basis_list[np.argmin(energies)], energies[np.argmin(energies)]])

min_basis, min_energy = min(basis, key=lambda x: x[1])
print("The basis with the lowest energy:", min_basis)
print("Lowest energy:", min_energy)

(1, 0, 0, 0, 0) 1.297316207738111
(1, 0, 0, 0, 0) 1.3126304888524412
(1, 0, 0, 0, 0) 1.5171734676872655
(1, 0, 0, 0, 0) 1.469967039299465
(1, 0, 0, 0, 0) 1.38829200318564
(1, 0, 0, 0, 0) 1.5081413841766604
(1, 0, 0, 0, 0) 1.3519897020654188
(1, 0, 0, 0, 0) 1.3208684584265717
(1, 0, 0, 0, 0) 1.4022682424160084
(1, 0, 0, 0, 0) 1.4051379617726103
(1, 0, 0, 0, 0) 1.421663932012863
(1, 0, 0, 0, 0) 1.3835581521837765
(1, 0, 0, 0, 0) 1.2714530094934757
(1, 0, 0, 0, 0) 1.391524992465935
(1, 0, 0, 0, 0) 1.3510625218681818
(1, 0, 0, 0, 0) 1.2886419380913998
(1, 0, 0, 0, 0) 1.4529287226893493
(1, 0, 0, 0, 0) 1.288923106537243
(1, 0, 0, 0, 0) 1.4161516304027515
(1, 0, 0, 0, 0) 1.3309690453976695
The basis with the lowest energy: (1, 0, 0, 0, 0)
Lowest energy: 1.2714530094934757


In [110]:
operator_pool = []
for i in range(num_qubits):
    #operator_pool.append(qml.Identity(i))
    #operator_pool.append(qml.PauliX(i))
    operator_pool.append(qml.PauliY(i))
    operator_pool.append(qml.PauliZ(i))


In [111]:
cz_pool = []

for control in range(num_qubits):
        for target in range(num_qubits):
            if control != target:
                cz_pool.append(qml.CZ(wires=[control, target]))

In [117]:
dev = qml.device("default.qubit", wires=num_qubits, shots=shots)
basis_state = min_basis
#basis_state = [0,0,0,0,0]

@qml.qnode(dev)
def circuit(times, op_list, try_cz=False, cz_wires=None, trotter_steps=1):

    #qml.BasisState(basis_state, wires=range(num_qubits))

    for i, op in enumerate(op_list):
        if type(op) == qml.CZ:
            qml.CZ(wires=op.wires)
        else:
            qml.ApproxTimeEvolution(op, time=times[i], n=trotter_steps)

    if try_cz:
        qml.CZ(wires=cz_wires)

    return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))


In [118]:
dev2 = qml.device("default.qubit", wires=num_qubits)
@qml.qnode(dev2)
def grad_circuit(times, operator_ham, op_list, op_params, trotter_steps=1):

    #qml.BasisState(basis_state, wires=range(num_qubits))
    param_index = 0
    for op in op_list:
        if type(op) == qml.CZ:
            qml.CZ(wires=op.wires)
        else:
            qml.ApproxTimeEvolution(op, time=op_params[param_index], n=trotter_steps)
            param_index +=1

    qml.ApproxTimeEvolution(operator_ham, time=times, n=trotter_steps)

    return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))


In [119]:
def compute_grad(operator, op_list, op_params):
    t = pnp.tensor(0.0, requires_grad=True)
    grad_fn = qml.grad(grad_circuit)
    grad = grad_fn(t, operator, op_list, op_params)
    return grad

In [120]:
def cost_function(times, op_list):

    times = pnp.tensor(times, requires_grad=True)
    energy = circuit(times, op_list)
    
    return energy

In [121]:
num_steps = 4
op_list = []
op_params = []

#variables
max_iter = 5000
strategy = "randtobest1bin"
tol = 1e-3
abs_tol = 1e-3
popsize = 20

for i in range(num_steps):

    print(f"step: {i}")

    grad_list = []

    for op in operator_pool:
        grad = compute_grad(op, op_list, op_params)
        grad_list.append(abs(grad))

    maxidx = np.argmax(grad_list)
    op_list.append(operator_pool[maxidx])

    # Generate Halton sequence
    num_dimensions = len(op_list)
    num_samples = popsize
    halton_sampler = Halton(d=num_dimensions)
    halton_samples = halton_sampler.random(n=num_samples)
    scaled_samples = 2 * np.pi * halton_samples

    bounds = [(0, 2 * np.pi) for _ in range(num_dimensions)]
    x0 = np.concatenate((op_params, np.array([np.random.random()*2*np.pi])))
    res = differential_evolution(cost_function,
                                    bounds,
                                    x0=x0,
                                    args=(op_list,),
                                    maxiter=max_iter,
                                    tol=tol,
                                    atol=abs_tol,
                                    strategy=strategy,
                                    popsize=popsize,
                                    init=scaled_samples,
                                    )
    
    if i!=0: pre_min_e = min_e
    min_e = res.fun
    pre_op_params = op_params
    op_params = res.x

    print(f"Min E: {min_e}")
    print(res.success)

    
    print("Testing CZ pool")
    cz_e = []
    for term in cz_pool:
        energy = circuit(op_params, op_list, try_cz=True, cz_wires=term.wires)
        cz_e.append(energy)

    min_cz_e = cz_e[np.argmin(cz_e)]
    min_cz_term = cz_pool[np.argmin(cz_e)]
    if min_cz_e < min_e:
        print(f"Adding {min_cz_term} reduces energy further")
        op_list.append(min_cz_term)
        min_e = min_cz_e
        print(f"Min E: {min_e}")
    
    if i!=0:
        if abs(pre_min_e - min_e) < 1e-8:
            print("gradient converged")
            op_list.pop()
            op_params = pre_op_params
            break


    

step: 0
Min E: 0.9540992997527721
False
Testing CZ pool
step: 1
Min E: 0.9428863331852235
False
Testing CZ pool
step: 2
Min E: 0.9409602834915831
False
Testing CZ pool
step: 3
Min E: 0.9383674929218713
False
Testing CZ pool


In [122]:
op_list

[Y(4), Y(3), Y(4), Y(4)]

In [46]:
dev = qml.device("default.qubit", wires=num_qubits, shots=2)
@qml.qnode(dev)
def final_circuit(params):

    qml.BasisState(basis_state, wires=range(num_qubits))
    params_index = 0
    for op in op_list:
        if type(op) == qml.CZ:
            qml.CZ(wires=op.wires)
        else:
            pauli_string = qml.pauli.pauli_word_to_string(op)
            qml.PauliRot(params[params_index], pauli_string, wires=op.wires)
            params_index += 1

    return qml.state()

In [47]:
x0 = np.random.uniform(0, 2 * np.pi, size=len(op_list))
print(qml.draw(final_circuit)(x0))

0: ─╭|Ψ⟩──RY(0.08)─┤  State
1: ─├|Ψ⟩──RY(3.95)─┤  State
2: ─├|Ψ⟩───────────┤  State
3: ─╰|Ψ⟩───────────┤  State


In [48]:
def overlap_function(params):

    params = pnp.tensor(params, requires_grad=True)
    ansatz_state = final_circuit(params)
    
    overlap = np.vdot(min_eigenvector, ansatz_state)
    cost = np.abs(overlap)**2  

    return (1 - cost)

In [49]:
bounds = [(0, 2 * np.pi) for _ in range(len(op_list))]

# Generate Halton sequence
num_dimensions = len(op_list)
num_samples = popsize
halton_sampler = Halton(d=num_dimensions)
halton_samples = halton_sampler.random(n=num_samples)
scaled_samples = 2 * np.pi * halton_samples

#variables
max_iter = 300
strategy = "randtobest1bin"
tol = 1e-6
abs_tol = 1e-5
popsize = 20

print("Running for overlap")
overlap_res = differential_evolution(overlap_function,
                                    bounds,
                                    maxiter=max_iter,
                                    tol=tol,
                                    atol=abs_tol,
                                    strategy=strategy,
                                    popsize=popsize,
                                    init=scaled_samples,
                                    )

Running for overlap


DeviceError: Measurement state(wires=[0, 1, 2, 3]) not accepted with finite shots on default.qubit

In [45]:
overlap_res.fun

np.float64(0.10755863889768569)

In [38]:
dev = qml.device("default.qubit", wires=num_qubits)
@qml.qnode(dev)
def energy_circuit(params):

    qml.BasisState(basis_state, wires=range(num_qubits))
    params_index = 0
    for op in op_list:
        if type(op) == qml.CZ:
            qml.CZ(wires=op.wires)
        else:
            pauli_string = qml.pauli.pauli_word_to_string(op)
            qml.PauliRot(params[params_index], pauli_string, wires=op.wires)
            params_index += 1

    return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))

In [39]:
x0 = overlap_res.x
#x0 = hf_res.x
energy_circuit(x0)

np.float64(8.067428176795758)

In [301]:
data = {"potential": potential,
        "cutoff": cutoff,
        "optimizer": "DE",
        "num steps": num_steps,
        "basis_state": basis_state,
        "op_list": [str(o) for o in op_list],
        "overlap": overlap_res.fun,
        "hellinger": hf_res.fun
        }

In [302]:
data

{'potential': 'AHO',
 'cutoff': 8,
 'optimizer': 'DE',
 'num steps': 5,
 'basis_state': [1, 0, 0, 0],
 'op_list': ['Y(2)'],
 'overlap': np.float64(0.001074857629434267),
 'hellinger': np.float64(0.0005375733072474631)}

In [303]:
# Save the variable to a JSON file
base_path = r"C:\Users\Johnk\Documents\PhD\Quantum Computing Code\Quantum-Computing\SUSY\SUSY QM\PennyLane\ADAPT-VQE\Files\TimeEv\\"
os.makedirs(base_path, exist_ok=True)
path = base_path + "{}_{}.json".format(potential, cutoff)
with open(path, 'w') as json_file:
    json.dump(data, json_file, indent=4)

In [88]:
#variables
max_iter = 10000
strategy = "randtobest1bin"
tol = 1e-3
atol = 1e-3
popsize = 20

num_steps = 5
op_list = []
op_params = []

for i in range(num_steps):

    print(f"step: {i}")

    grad_list = []

    for op in operator_pool:
        grad = compute_grad(op, op_list, op_params)
        grad_list.append(abs(grad))

    maxidx = np.argmax(grad_list)
    op_list.append(operator_pool[maxidx])

    bounds = [(0, 2 * np.pi) for _ in range(len(op_list))]
    res = differential_evolution(cost_function,
                                    bounds,
                                    args=(op_list,),
                                    maxiter=max_iter,
                                    tol=tol,
                                    atol=atol,
                                    strategy=strategy,
                                    popsize=popsize
                                    )
    if i!=0: pre_min_e = min_e
    min_e = res.fun
    pre_op_params = op_params
    op_params = res.x

    print(f"Min E: {min_e}")
    print(res.success)

    print("Testing CZ pool")
    cz_e = []
    for term in cz_pool:
        energy = circuit(op_params, op_list, try_cz=True, cz_wires=term.wires)
        cz_e.append(energy)

    min_cz_e = cz_e[np.argmin(cz_e)]
    min_cz_term = cz_pool[np.argmin(cz_e)]
    if min_cz_e < min_e:
        print(f"Adding {min_cz_term} reduces energy further")
        op_list.append(min_cz_term)
        min_e = min_cz_e
        print(f"Min E: {min_e}")
    
    if i!=0:
        if abs(pre_min_e - min_e) < 1e-8:
            print("gradient converged")
            op_list.pop()
            op_params = pre_op_params
            break


    

step: 0
Min E: 21.835191043096877
True
Testing CZ pool
step: 1
Min E: 13.952265530952426
True
Testing CZ pool
step: 2
Min E: 12.930673281831922
True
Testing CZ pool
step: 3
Min E: 1.090499530121596
True
Testing CZ pool
Adding CZ(wires=[3, 4]) reduces energy further
Min E: 1.0337017717049055
step: 4
Min E: 0.9164451094702184
True
Testing CZ pool
